In [1]:
import math

def calcula_prob(v, u):
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna um vetor p = [p1, p2] com a prob do jogador 1 ganhar do 2 e do 2 ganhar do 1
    p = [0, 0]
    p[0] = math.exp(u[0]*v[1])/(math.exp(u[0]*v[1]) + math.exp(u[1]*v[0]))
    p[1] = 1 - p[0]
    
    return p

def gera_parametro(p):
    # p = [p0, p1], onde p0 = x/(x + y) e p1 = y/(x + y), com x = e^(u0*v1) e y = e^(v0*u1)
    # queremos encontrar v = [v0, v1] e u = [u0, u1]
    x = math.log(p[0]) # faz x = ln(x) - ln(x + y) = ln(x) = u0*v1
    y = math.log(p[1]) # faz y = ln(y) - ln(x + y) = ln(y) = v0*u1
    
    # considerando u0 = 1 = v0, temos que 
    u = [1, x]
    v = [1, y]
    
    '''
    tentativa de normalizar
    
    v = [1/math.sqrt(1 + x**2), x/math.sqrt(1 + x**2)]
    u = [1/math.sqrt(1 + y**2), y/math.sqrt(1 + y**2)]
    '''
    
    return [u, v]

In [2]:
v1 = [   1,               0]
v2 = [-0.5,  math.sqrt(3)/2]
v3 = [-0.5, -math.sqrt(3)/2]

print("Jogo entre v1 e v2:", calcula_prob(v1, v2))
print("Jogo entre v2 e v3:", calcula_prob(v2, v3))
print("Jogo entre v3 e v1:", calcula_prob(v3, v1))

Jogo entre v1 e v2: [0.2960820052793571, 0.7039179947206429]
Jogo entre v2 e v3: [0.2960820052793571, 0.7039179947206429]
Jogo entre v3 e v1: [0.2960820052793571, 0.7039179947206429]


In [3]:
p = calcula_prob(v1, v2)
param = gera_parametro(p)
q = calcula_prob(param[0], param[1])
print("Probabilidades com p:", p)
print("Parâmetros:", param)
print("Probabilidades os parâmetros encontrados:", q)

Probabilidades com p: [0.2960820052793571, 0.7039179947206429]
Parâmetros: [[1, -1.2171188181652253], [1, -0.3510934143807867]]
Probabilidades os parâmetros encontrados: [0.2960820052793571, 0.7039179947206429]


In [4]:
import math

def calcula_prob(v, u):
    # calcula a probabilidade de acordo com a ideia do logit
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna um vetor p = [p1, p2] com a prob do jogador 1 ganhar do 2 e do 2 ganhar do 1
    logit = v[1]/u[0] - u[1]/v[0]
    p = [0, 0]
    p[0] = 1/(1 + math.exp(-logit))
    p[1] = 1 - p[0]
    
    return p

def verossimilhanca(resultados, jogadores):
    # faz o cálculo da verossimilhança negativa dado como entrada uma lista
    # de resultados e uma lista de parâmetros, conforme abaixo
    
    # resultados é uma lista de lista/matriz n x 2, onde cada lista/linha
    # representa um jogo, sendo da forma [ID_w, ID_l]
    # jogadores é a lista de parâmetros dos jogadores, assim parâmetros[0]
    # contém os parâmetros do jogador de ID = 0
    
    log_ver = 0
    p = [0, 0]
    
    for i in range(len(resultados)):
        p = calcula_prob(jogadores[resultados[i][0]], jogadores[resultados[i][1]])
        log_ver += math.log(p[0])
    
    return - log_ver

In [5]:
P1 = [   1,               0]
P2 = [-0.5,  math.sqrt(3)/2]
P3 = [-0.5, -math.sqrt(3)/2]

print("Jogo entre P1 e P2:", calcula_prob(P1, P2))
print("Jogo entre P2 e P3:", calcula_prob(P2, P3))
print("Jogo entre P3 e P1:", calcula_prob(P3, P1))

Jogo entre P1 e P2: [0.2960820052793571, 0.7039179947206429]
Jogo entre P2 e P3: [0.030351090329424395, 0.9696489096705756]
Jogo entre P3 e P1: [0.2960820052793571, 0.7039179947206429]


In [6]:
resultados = [[0, 1], [1, 2], [2, 0], [2, 1]]
jogadores = [P1, P2, P3]

verossimilhanca(resultados, jogadores)

5.959981695062614

In [7]:
def calcula_prob(v, u):
    # calcula a probabilidade de acordo com a ideia do logit
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna p, a prob do jogador 1 ganhar do jogador 2
    logit = v[1]/u[0] - u[1]/v[0]
    
    # coloquei esse try pois tem vezes que essa exponencial resulta em um valor muito grande
    # gerando o OverflowError
    try:
        p = 1/(1 + math.exp(-logit))
    except OverflowError:
        p = float(1e-31)
    
    return p

def verossimilhanca(jogadores, resultados):
    # faz o cálculo da verossimilhança negativa dado como entrada uma lista
    # de resultados e uma lista de parâmetros, conforme abaixo
    
    # jogadores é a lista de parâmetros dos jogadores. Nela os parâmetros
    # do jogador de ID = i estão em parâmetros[2*i] e em parâmetros[2*i + 1]
    # resultados é uma lista de lista/matriz n x 2, onde cada lista/linha
    # representa um jogo, sendo da forma [ID_w, ID_l]
    
    log_ver = 0
    
    for i in range(len(resultados)):
        indice_1 = 2*resultados[i][0]
        indice_2 = 2*resultados[i][1]
        param_1 = [jogadores[indice_1], jogadores[indice_1 + 1]]
        param_2 = [jogadores[indice_2], jogadores[indice_2 + 1]]
        p = calcula_prob(param_1, param_2)
        log_ver += math.log(p)
    
    return - log_ver

In [8]:
resultados = [[0, 1], [1, 2], [2, 0], [2, 1]]
jogadores = [1, 0, -0.5, math.sqrt(3)/2, -0.5, -math.sqrt(3)/2]

verossimilhanca(jogadores, resultados)

5.959981695062614

In [9]:
cons = ({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[0]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[4]})

In [10]:
from scipy.optimize import minimize

results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)

In [11]:
results.x = results.x * (1/min(abs(results.x)))
print("Os parâmetros encontrados são:\n", results.x)

Os parâmetros encontrados são:
 [505.70472945 251.77736851  40.77817833 -42.68048805 551.18195062
   1.        ]


In [12]:
results.fun

2.3369976685748237

In [13]:
cons = []
jogadores = []
for i in range(3):
    cons.append({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2*i]})
    jogadores.append(1)
    jogadores.append(0)

In [14]:
cons = tuple(cons)

In [15]:
results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)

In [16]:
results.x

array([ 2.99009532e-06,  1.69032020e-07,  1.67042410e+00, -3.08439143e-01,
        1.67042279e+00,  3.08440896e-01])

In [17]:
results.fun

1.4201971731668042

In [18]:
from openpyxl import load_workbook
import numpy as np

def pega_jogos(file):
    games = load_workbook(file)

    jogos = games['Planilha1']

    resultados = []

    for row in jogos.iter_rows():
        jogo = []
        for cell in row:
            if cell.value != 'w' and cell.value != 'l':
                jogo.append(cell.value)
        resultados.append(jogo)
    
    if resultados[0] == []:
        resultados.pop(0)
    return resultados

def conta_jogadores(resultados):
    contagem = 0
    for i in range(len(resultados)):
        for j in resultados[i]:
            if j > contagem:
                contagem = j
    
    return contagem + 1

def cria_restricoes(contagem):
    restricoes = []
    for i in range(contagem):
        restricoes.append({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2*i]})
    
    restricoes = tuple(restricoes)
    return restricoes

def cria_jogadores(contagem):
    jogadores = []
    for i in range(contagem):
        jogadores.append(np.random.random())
        jogadores.append(np.random.random())
    
    return jogadores

def calcula_parametros(file):
    resultados = pega_jogos(file)
    contagem = conta_jogadores(resultados)
    cons = cria_restricoes(contagem)
    jogadores = cria_jogadores(contagem)
    results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)
    
    return results

In [ ]:
results = calcula_parametros('matches.xlsx')

In [ ]:
results.fun

In [27]:
resultados = pega_jogos("matches.xlsx")

In [28]:
len(resultados)

11497